## Imports

In [1]:
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

## Init **Roads** DF

In [2]:
data = [
    ['11ου Συντάγματος', 'residential', '[{"lat": 37.5163202, "lon": 22.3670926}, {"lat": 37.5163008, "lon": 22.3671941}, {"lat": 37.5162872, "lon": 22.3672417}, {"lat": 37.5160483, "lon": 22.3678892}, {"lat": 37.5160213, "lon": 22.3679662}, {"lat": 37.5159004, "lon": 22.3683196}, {"lat": 37.5158288, "lon": 22.368505}, {"lat": 37.5157759, "lon": 22.3686216}, {"lat": 37.5157207, "lon": 22.3687294}, {"lat": 37.5156562, "lon": 22.3688413}, {"lat": 37.5155724, "lon": 22.3689578}, {"lat": 37.5154084, "lon": 22.3691304}]'],
    ['11ου Συντάγματος', 'residential', '[{"lat": 37.516339, "lon": 22.366991}, {"lat": 37.5163202, "lon": 22.3670926}]']
]


filename_omp_streets = "../../DataSets/API_Responses/OSM/OSM_roads.csv"


# Convert data to DataFrame
#df = pd.DataFrame(data, columns=["name", "highway", "coordinates"])

df_streets = pd.read_csv(filename_omp_streets, )

# List to store the new rows
new_rows = []

# Process each row in the DataFrame
for idx, row in df_streets.iterrows():
    # Parse the coordinates column
    coordinates = ast.literal_eval(row['coordinates'])  # Convert string of list to actual list of dicts
    
    # Iterate over the consecutive coordinate pairs (lat-start, lon-start) and (lat-end, lon-end)
    for i in range(len(coordinates) - 1):
        start = coordinates[i]
        end = coordinates[i + 1]
        
        # Define the required values for the new DataFrame
        lat_start = start['lat']
        lon_start = start['lon']
        lat_end = end['lat']
        lon_end = end['lon']
        
        # Calculate the minimum latitude and midpoint longitude
        lat_min = min(lat_start, lat_end)
        lon_mid = (lon_start + lon_end) / 2
        
        # Append the new row with the calculated values and original information
        new_rows.append({
            'lat_start': lat_start,
            'lon_start': lon_start,
            'lat_end': lat_end,
            'lon_end': lon_end,
            'lat_mid': lat_min,
            'lon_mid': lon_mid,
            'name': row['name'],
            'highway': row['highway'],
            'original_row_id': idx  # The index can serve as the original row id
        })

# Create a new DataFrame from the list of new rows
df_roads_segments = pd.DataFrame(new_rows)

# Display the new DataFrame
print(df_roads_segments)


       lat_start  lon_start    lat_end    lon_end    lat_mid    lon_mid  \
0      37.516320  22.367093  37.516301  22.367194  37.516301  22.367143   
1      37.516301  22.367194  37.516287  22.367242  37.516287  22.367218   
2      37.516287  22.367242  37.516048  22.367889  37.516048  22.367565   
3      37.516048  22.367889  37.516021  22.367966  37.516021  22.367928   
4      37.516021  22.367966  37.515900  22.368320  37.515900  22.368143   
...          ...        ...        ...        ...        ...        ...   
34391  37.510063  22.367360  37.510113  22.367401  37.510063  22.367380   
34392  37.510113  22.367401  37.510173  22.367437  37.510113  22.367419   
34393  37.510173  22.367437  37.510231  22.367462  37.510173  22.367450   
34394  37.510231  22.367462  37.510295  22.367479  37.510231  22.367471   
34395  37.510204  22.373743  37.510549  22.373441  37.510204  22.373592   

                   name      highway  original_row_id  
0      11ου Συντάγματος  residential       

## Define **Haversine Disatnce**

In [3]:
def haversine_distance(x, y):
    # Convert degrees to radians
    x = np.radians(x)
    y = np.radians(y)
    
    # Compute differences in latitudes and longitudes
    dlat = y[0] - x[0]
    dlon = y[1] - x[1]
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(x[0]) * np.cos(y[0]) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers
    r = 6371
    
    return r * c


# Example usage
coords_start = np.array([[37.5163202, 22.3670926], [37.5163008, 22.3671941]])  # (lat, lon) for start
coords_end = np.array([[37.5162872, 22.3672417], [37.5160483, 22.3678892]])    # (lat, lon) for end

distances = haversine_distance(coords_start, coords_end)
print(distances)


[0.02257018 0.0733741 ]


## Init **DataSets DF**

In [17]:
import pandas as pd
import ast

# Load vehicle data
filename_coords = "../../DataSets/API_Responses/Vehicle_Data/all_vehicle_responses.csv"
df_coords = pd.read_csv(filename_coords)


df_coords["coordinates"] = df_coords.apply(lambda row: [(row["lat"], row["lng"])], axis=1)
print(df_coords["coordinates"])




0        [(38.0344583, 23.7481766)]
1        [(38.0345983, 23.7481433)]
2        [(38.0346333, 23.7481316)]
3        [(38.0346666, 23.7481633)]
4        [(38.0347133, 23.7481216)]
                    ...            
54493      [(37.53146, 22.3697683)]
54494     [(37.531275, 22.3694733)]
54495    [(37.5311216, 22.3691949)]
54496    [(37.5311483, 22.3689666)]
54497    [(37.5312433, 22.3687049)]
Name: coordinates, Length: 54498, dtype: object


## **NearestNeighbors**

In [21]:
import numpy as np
from sklearn.neighbors import NearestNeighbors


# Create NearestNeighbors model with BallTree for indexing and custom metric
neighbors = NearestNeighbors(n_neighbors=50, metric=haversine_distance, algorithm='ball_tree')
neighbors.fit(df_roads_segments[['lat_mid', 'lon_mid']].values)


# Example query point
query_point = np.array([37.5311483, 22.3689666])

# Find the closest point to the query point
# dist, ind = neighbors.kneighbors(query_point)
dist, ind = neighbors.kneighbors(query_point.reshape(1, -1))


print("Closest point index:", ind)
print("Distance (in meters) to closest point:", dist * 111,000) #! 1 degree (coordinate) == 11,000 meters



Closest point index: [[ 6849  6768  6848  6850  6854  6855  6767  6769  6766  6856  6851  6765
   6770  6857  6852  6771  6764  6858  6853  6772  6859  6773  6860  6774
   6861  6775  6776  6777  6778  6862  6779  6780  6781  6782  6783  6784
   6785  6786  6787 21853  6788 21852  6789 21851  6792  6793  6790  6791
   6794  6795]]
Distance (in meters) to closest point: [[ 0.11226514  0.94036491  0.94413688  1.07020172  1.26339783  1.29166383
   1.31871887  1.32710952  1.60944126  1.74874737  1.87469111  2.0269679
   2.18247387  2.65325758  2.69469493  3.22287596  3.52265754  3.61603543
   4.07854649  4.16525589  4.67267553  5.29932684  5.72012485  6.49440983
   7.90136116  7.9428927   9.00580398  9.93344716 10.70156064 11.13414306
  11.13477197 11.45358935 12.00534336 12.65783095 13.18394798 13.71725591
  14.1889502  14.51214068 14.87561791 14.90034235 14.92973248 15.08551753
  15.38677538 15.49132226 15.67811212 15.73036133 15.82594619 15.87156625
  15.92643522 16.28697025]] 0


In [6]:
import pandas as pd
import ast

# Load vehicle data
filename_coords = "../../DataSets/API_Responses/Vehicle_Data/all_vehicle_responses.csv"
df_coords = pd.read_csv(filename_coords)


df_coords["coordinates"] = df_coords.apply(lambda row: [(row["lat"], row["lng"])], axis=1)
print(df_coords["coordinates"])


# Load building data
filename_buildings = "../../DataSets/API_Responses/OSM/OSM_houses.csv"
df_buildings = pd.read_csv(filename_buildings)

# Convert building coordinates from string to a list of (lat, lon) tuples
df_buildings["coordinates"] = df_buildings["coordinates"].apply(
    lambda x: [(point["lat"], point["lon"]) for point in ast.literal_eval(x)]
)
print(df_buildings["coordinates"])


0        [(38.0344583, 23.7481766)]
1        [(38.0345983, 23.7481433)]
2        [(38.0346333, 23.7481316)]
3        [(38.0346666, 23.7481633)]
4        [(38.0347133, 23.7481216)]
                    ...            
54493      [(37.53146, 22.3697683)]
54494     [(37.531275, 22.3694733)]
54495    [(37.5311216, 22.3691949)]
54496    [(37.5311483, 22.3689666)]
54497    [(37.5312433, 22.3687049)]
Name: coordinates, Length: 54498, dtype: object
0      [(37.5153299, 22.3878963), (37.5152124, 22.387...
1      [(37.5157459, 22.3877368), (37.5156284, 22.387...
2      [(37.5109022, 22.3694617), (37.511067, 22.3695...
3      [(37.5106904, 22.3636614), (37.5105267, 22.363...
4      [(37.5123545, 22.3841353), (37.5121766, 22.385...
                             ...                        
300    [(37.5021164, 22.4977753), (37.5021997, 22.497...
301    [(37.5062436, 22.50127), (37.5064016, 22.50129...
302    [(37.5272306, 22.3712629), (37.5273966, 22.372...
303    [(37.5299764, 22.375622), (37.529875

## **Project Points to Segments**

In [7]:
import numpy as np




def project_points_to_segments(px, py, x1, y1, x2, y2):
    """
    Projects multiple points (px, py) onto multiple line segments defined by (x1, y1) -> (x2, y2).
    If a point falls outside the vertical range of its segment, it remains unchanged.
    
    Inputs:
        - px, py: Arrays of shape (N,) representing point coordinates
        - x1, y1, x2, y2: Arrays of shape (N,) representing segment endpoints
    
    Output:
        - proj_x, proj_y: Arrays of shape (N,) with projected points
    """
    # Compute segment vectors and point vectors
    vx = x2 - x1  # Segment direction x-component
    vy = y2 - y1  # Segment direction y-component
    wx = px - x1  # Vector from segment start to point x-component
    wy = py - y1  # Vector from segment start to point y-component

    # Compute dot products
    dot_vv = vx * vx + vy * vy  # Dot product of v with itself
    dot_wv = wx * vx + wy * vy  # Dot product of w with v

    # Avoid division by zero (if the segment is a single point)
    mask_zero_length = dot_vv == 0
    t = np.zeros_like(dot_wv, dtype=np.float64)  # Explicitly specify float type for 't'
    np.divide(dot_wv, dot_vv, out=t, where=~mask_zero_length)  # Perform division


    # Clamp t between 0 and 1 to stay within the segment
    t = np.clip(t, 0, 1)

    # Compute projected points
    proj_x = x1 + t * vx
    proj_y = y1 + t * vy

    # If a point is outside the vertical range of its segment, leave it unchanged
    mask_outside_vertical = (py < np.minimum(y1, y2)) | (py > np.maximum(y1, y2))
    proj_x = np.where(mask_outside_vertical, px, proj_x)
    proj_y = np.where(mask_outside_vertical, py, proj_y)

    return proj_x, proj_y

# Example Usage with Batched Inputs
px = np.array([3, 6, 2])  # Points to project
py = np.array([4, 5, 8])
x1 = np.array([1, 5, 0])  # Segment start points
y1 = np.array([2, 4, 1])
x2 = np.array([5, 7, 3])  # Segment end points
y2 = np.array([6, 5, 4])

proj_x, proj_y = project_points_to_segments(px, py, x1, y1, x2, y2)
print("Projected X:", proj_x)
print("Projected Y:", proj_y)


Projected X: [3.  6.2 2. ]
Projected Y: [4.  4.6 8. ]


In [8]:
import numpy as np

def latlon_to_cartesian(lat, lon):
    """Converts lat-long to 3D Cartesian coordinates on a unit sphere."""
    lat, lon = np.radians(lat), np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.stack([x, y, z], axis=-1)  # Shape (N, 3)

def cartesian_to_latlon(cart):
    """Converts 3D Cartesian coordinates back to latitude and longitude."""
    x, y, z = cart[..., 0], cart[..., 1], cart[..., 2]
    lat = np.arcsin(z)  # Inverse sine for latitude
    lon = np.arctan2(y, x)  # Arctan2 for longitude
    return np.degrees(lat), np.degrees(lon)

def project_geodesic_points(px, py, x1, y1, x2, y2):
    """
    Projects lat-long points (px, py) onto geodesic segments (x1, y1) to (x2, y2).
    Leaves the point unchanged if it's outside the segment range.

    Inputs:
        - px, py: Arrays of shape (N,) with latitude and longitude of points to project.
        - x1, y1, x2, y2: Arrays of shape (N,) defining segment endpoints in lat-long.

    Returns:
        - proj_lat, proj_lon: Arrays of shape (N,) with projected coordinates.
    """
    
    # Convert lat-lon to Cartesian 3D coordinates
    P = latlon_to_cartesian(px, py)  # Shape (N, 3)
    A = latlon_to_cartesian(x1, y1)
    B = latlon_to_cartesian(x2, y2)

    # Compute the great-circle normal (cross product of A and B)
    N = np.cross(A, B)  # Shape (N, 3)
    N /= np.linalg.norm(N, axis=-1, keepdims=True)  # Normalize

    # Compute projection of P onto the great-circle plane
    P_proj = P - np.sum(P * N, axis=-1, keepdims=True) * N
    P_proj /= np.linalg.norm(P_proj, axis=-1, keepdims=True)  # Normalize to stay on unit sphere

    # Convert projected Cartesian points back to lat-lon
    proj_lat, proj_lon = cartesian_to_latlon(P_proj)

    # Check if projected points are within segment bounds using angular distance
    dot_AB = np.sum(A * B, axis=-1)  # Cosine of angle between A and B
    dot_AP = np.sum(A * P_proj, axis=-1)  # Cosine of angle between A and projected point
    dot_BP = np.sum(B * P_proj, axis=-1)  # Cosine of angle between B and projected point

    # If the projection falls outside the segment, snap to the nearest endpoint
    mask_before_A = dot_AP < dot_AB  # If P_proj is before A
    mask_after_B = dot_BP < dot_AB   # If P_proj is after B

    proj_lat = np.where(mask_before_A, x1, proj_lat)
    proj_lon = np.where(mask_before_A, y1, proj_lon)
    proj_lat = np.where(mask_after_B, x2, proj_lat)
    proj_lon = np.where(mask_after_B, y2, proj_lon)

    return proj_lat, proj_lon



# Example Usage with Batched Inputs
px = np.array([37.5, 38.0, 37.8])  # Points to project
py = np.array([22.3, 22.5, 22.6])
x1 = np.array([37.516, 37.4, 37.7])  # Segment start points
y1 = np.array([22.367, 22.2, 22.5])
x2 = np.array([37.520, 37.6, 37.9])  # Segment end points
y2 = np.array([22.370, 22.4, 22.7])

proj_lat, proj_lon = project_geodesic_points(px, py, x1, y1, x2, y2)

print("Projected Latitudes:", proj_lat)
print("Projected Longitudes:", proj_lon)


Projected Latitudes: [37.52       37.6        37.80006828]
Projected Longitudes: [22.37       22.4        22.59989064]
